# Próba wykorzystania istniejącego modelu RegGNN
https://github.com/basiralab/RegGNN/blob/main/proposed_method/RegGNN.py

Nasze zadanie opiera się na przygotowaniu modelu rozwiązującego zadanie regresji grafowej, czyli na podstawie grafów, ich cech globalnych i etykiet chcemy prognozować wartość metryki średniej ilości wykorzystanych transceiverów podczas symulacji.

In [1]:
import torch

# Wczytaj zbiór uczący
train_dataset = torch.load('train_dataset.pt')
# wczytaj zbiór testowy
test_dataset = torch.load('test_dataset.pt')

print(len(train_dataset))
print(len(test_dataset))

184
46


# Przykładowy model regresyjny Grafowej sieci neuronowej GNN

In [2]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch_geometric.nn import GINConv, global_add_pool

class GINRegression(nn.Module):
    def __init__(self, input_dim, hidden_dim, output_dim, num_layers, dropout):
        super(GINRegression, self).__init__()

        # GINConv layers
        self.conv1 = GINConv(nn.Sequential(
            nn.Linear(input_dim, hidden_dim),
            nn.ReLU(),
            nn.Linear(hidden_dim, hidden_dim)
        ), train_eps=True)

        self.convs = nn.ModuleList()
        for _ in range(num_layers - 1):
            self.convs.append(GINConv(nn.Sequential(
                nn.Linear(hidden_dim, hidden_dim),
                nn.ReLU(),
                nn.Linear(hidden_dim, hidden_dim)
            ), train_eps=True))

        # Fully connected layers for regression
        self.fc1 = nn.Linear(hidden_dim + 2, hidden_dim)  # Dodane 2 na cechy globalne
        self.fc2 = nn.Linear(hidden_dim, output_dim)

        self.dropout = dropout

    def forward(self, data):
        x, edge_index, batch = data.x, data.edge_index, data.batch
        density, avg_clustering = data.global_feature[:, 0], data.global_feature[:, 1]  # Wyciąganie globalnych cech

        # GINConv layers
        x = F.relu(self.conv1(x, edge_index))
        for conv in self.convs:
            x = F.relu(conv(x, edge_index))

        # Global pooling and fully connected layers with global features
        x = global_add_pool(x, batch)
        x = torch.cat([x, density.view(-1, 1), avg_clustering.view(-1, 1)], dim=1)  # Dodane globalne cechy
        x = F.relu(self.fc1(x))
        x = F.dropout(x, p=self.dropout, training=self.training)
        x = self.fc2(x)

        return x
    
    def loss(self, pred, score):
        return F.mse_loss(pred, score)

In [3]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch_geometric.nn import GraphConv

class GraphRegressionModel(nn.Module):
    def __init__(self, num_node_features=1, hidden_dim=64, output_dim=1, dropout=0.5):
        super(GraphRegressionModel, self).__init__()

        # Graph Convolutional Layer
        self.conv1 = GraphConv(num_node_features, hidden_dim)

        # Fully Connected Layers with Dropout
        self.fc1 = nn.Linear(hidden_dim + 2, hidden_dim)
        self.dropout = nn.Dropout(p=dropout)
        self.fc2 = nn.Linear(hidden_dim, output_dim)

    def forward(self, data):
        x, edge_index, edge_weight, global_feature = data.x, data.edge_index, data.edge_weight, data.global_feature

        # Apply Graph Convolution
        x = self.conv1(x, edge_index, edge_weight=edge_weight)

        # Global features concatenation
        global_feature = global_feature.expand(x.size(0), -1)  # Dostosuj global_feature do rozmiaru x
        x = torch.cat([x, global_feature], dim=1)

        # Fully Connected Layers with Dropout
        x = F.relu(self.fc1(x))
        x = self.dropout(x)
        x = self.fc2(x)

        return x
    
    def loss(self, pred, score):
        return F.mse_loss(pred, score)

In [38]:
from hyperopt import fmin, tpe, hp

# Zdefiniuj funkcję oceny (score function)
def objective(params):
    hidden_dim = int(params['hidden_dim'])
    dropout = params['dropout']

    model = GraphRegressionModel(hidden_dim=hidden_dim, dropout=dropout)
    
    # Definiuj optymalizator, liczbę epok, itp.
    optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
    num_epochs = 200
    
    for epoch in range(num_epochs):
        # Trening i ewaluacja modelu
        for data in train_loader:
            optimizer.zero_grad()
            output = model(data)
            loss = model.loss(output, data.y.view(-1, 1).float())
            loss.backward()
            optimizer.step()
    
    # Ewaluacja na zbiorze testowym
    test_loss = 0.0  
    for data in test_loader:
        output = model(data)
        test_loss += model.loss(output, data.y.view(-1, 1).float()).item()
    
    # Zwróć funkcję oceny (score)
    return test_loss

# Przestrzeń poszukiwań dla hyperopt
space = {
    'hidden_dim': hp.quniform('hidden_dim', 32, 256, 32),  # Przeszukuj wartości co 32
    'dropout': hp.uniform('dropout', 0.05, 0.8),
}

# Minimalizacja funkcji oceny za pomocą algorytmu TPE
best = fmin(fn=objective, space=space, algo=tpe.suggest, max_evals=10)  # Dla przykładu ustawiono max_evals na 10
print("Najlepsze hiperparametry:", best)

  0%|          | 0/10 [00:00<?, ?trial/s, best loss=?]

C:\Users\jderd\AppData\Local\Temp\ipykernel_21620\794946686.py:36: UserWarning: Using a target size (torch.Size([1, 1])) that is different to the input size (torch.Size([28, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(pred, score)



100%|██████████| 10/10 [11:39<00:00, 69.90s/trial, best loss: 53519.48545837402]
Najlepsze hiperparametry: {'dropout': 0.06769502094656675, 'hidden_dim': 192.0}


In [39]:
import torch
from torch_geometric.data import DataLoader

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Ustawienie optymalnych hiperparametrów
optimal_hidden_dim = int(best['hidden_dim'])
optimal_dropout = best['dropout']

# Inicjalizacja modelu z optymalnymi hiperparametrami
optimal_model = GraphRegressionModel(hidden_dim=optimal_hidden_dim, dropout=optimal_dropout)

# Definiuj DataLoader dla zbiorów uczącego i testowego
train_loader = DataLoader(train_dataset, batch_size=1, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=1, shuffle=False)

# Definiuj optymalizator, liczbę epok, itp.
optimizer = torch.optim.Adam(optimal_model.parameters(), lr=0.001)
num_epochs = 200

# Listy do śledzenia train loss i test loss
train_losses = []
test_losses = []

# Trening modelu
for epoch in range(num_epochs):
    print("Epoch:", epoch)
    for data in train_loader:
        optimizer.zero_grad()
        output = optimal_model(data)
        loss = optimal_model.loss(output, data.y.view(-1, 1).float())
        loss.backward()
        optimizer.step()

sample_data = test_dataset[0]

# Przeprowadzenie predykcji na przykładowym obiekcie
optimal_model.eval()
with torch.no_grad():
    prediction = optimal_model(sample_data)[0][0].item()

# Wyświetlenie wyników
print("Przewidywana wartość:", prediction)
print("Rzeczywista wartość:", sample_data.y.item())

Epoch: 0


C:\Users\jderd\AppData\Local\Temp\ipykernel_21620\794946686.py:36: UserWarning: Using a target size (torch.Size([1, 1])) that is different to the input size (torch.Size([28, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(pred, score)


Epoch: 1
Epoch: 2
Epoch: 3
Epoch: 4
Epoch: 5
Epoch: 6
Epoch: 7
Epoch: 8
Epoch: 9
Epoch: 10
Epoch: 11
Epoch: 12
Epoch: 13
Epoch: 14
Epoch: 15
Epoch: 16
Epoch: 17
Epoch: 18
Epoch: 19
Epoch: 20
Epoch: 21
Epoch: 22
Epoch: 23
Epoch: 24
Epoch: 25
Epoch: 26
Epoch: 27
Epoch: 28
Epoch: 29
Epoch: 30
Epoch: 31
Epoch: 32
Epoch: 33
Epoch: 34
Epoch: 35
Epoch: 36
Epoch: 37
Epoch: 38
Epoch: 39
Epoch: 40
Epoch: 41
Epoch: 42
Epoch: 43
Epoch: 44
Epoch: 45
Epoch: 46
Epoch: 47
Epoch: 48
Epoch: 49
Epoch: 50
Epoch: 51
Epoch: 52
Epoch: 53
Epoch: 54
Epoch: 55
Epoch: 56
Epoch: 57
Epoch: 58
Epoch: 59
Epoch: 60
Epoch: 61
Epoch: 62
Epoch: 63
Epoch: 64
Epoch: 65
Epoch: 66
Epoch: 67
Epoch: 68
Epoch: 69
Epoch: 70
Epoch: 71
Epoch: 72
Epoch: 73
Epoch: 74
Epoch: 75
Epoch: 76
Epoch: 77
Epoch: 78
Epoch: 79
Epoch: 80
Epoch: 81
Epoch: 82
Epoch: 83
Epoch: 84
Epoch: 85
Epoch: 86
Epoch: 87
Epoch: 88
Epoch: 89
Epoch: 90
Epoch: 91
Epoch: 92
Epoch: 93
Epoch: 94
Epoch: 95
Epoch: 96
Epoch: 97
Epoch: 98
Epoch: 99
Epoch: 100
Epoch: 1

In [42]:
sample_data = test_dataset[30]

# Przeprowadzenie predykcji na przykładowym obiekcie
optimal_model.eval()
with torch.no_grad():
    prediction = optimal_model(sample_data)[0][0].item()

# Wyświetlenie wyników
print("Przewidywana wartość:", prediction)
print("Rzeczywista wartość:", sample_data.y.item())

Przewidywana wartość: 830.5301513671875
Rzeczywista wartość: 860.9299926757812


In [43]:
sample_data = test_dataset[9]

# Przeprowadzenie predykcji na przykładowym obiekcie
optimal_model.eval()
with torch.no_grad():
    prediction = optimal_model(sample_data)[0][0].item()

# Wyświetlenie wyników
print("Przewidywana wartość:", prediction)
print("Rzeczywista wartość:", sample_data.y.item())

Przewidywana wartość: 287.41595458984375
Rzeczywista wartość: 286.3299865722656


In [45]:
total_accuracy = 0.0

# Iteracja przez cały zbiór testowy
for i, sample_data in enumerate(test_dataset):
    # Przeprowadzenie predykcji na przykładowym obiekcie
    optimal_model.eval()
    with torch.no_grad():
        prediction = optimal_model(sample_data)[0][0].item()
    
    # Wyświetlenie wyników
    print(f"Przykład {i + 1}:")
    print("Przewidywana wartość:", prediction)
    print("Rzeczywista wartość:", sample_data.y.item())
    
    # Obliczenie procentowej jakości regresji
    actual_value = sample_data.y.item()
    accuracy = 100 * (1 - abs(prediction - actual_value) / actual_value)
    total_accuracy += accuracy
    
    print(f"Procentowa jakość regresji: {accuracy:.2f}%\n")

# Obliczenie średniej jakości procentowej
average_accuracy = total_accuracy / len(test_dataset)
print(f"Średnia procentowa jakość regresji dla wszystkich przykładów: {average_accuracy:.2f}%")


Przykład 1:
Przewidywana wartość: 834.1890869140625
Rzeczywista wartość: 874.5499877929688
Procentowa jakość regresji: 95.38%

Przykład 2:
Przewidywana wartość: 576.5999145507812
Rzeczywista wartość: 569.8499755859375
Procentowa jakość regresji: 98.82%

Przykład 3:
Przewidywana wartość: 461.2392578125
Rzeczywista wartość: 450.54998779296875
Procentowa jakość regresji: 97.63%

Przykład 4:
Przewidywana wartość: 881.8521118164062
Rzeczywista wartość: 839.4600219726562
Procentowa jakość regresji: 94.95%

Przykład 5:
Przewidywana wartość: 702.5387573242188
Rzeczywista wartość: 705.010009765625
Procentowa jakość regresji: 99.65%

Przykład 6:
Przewidywana wartość: 687.1488037109375
Rzeczywista wartość: 697.9400024414062
Procentowa jakość regresji: 98.45%

Przykład 7:
Przewidywana wartość: 870.410400390625
Rzeczywista wartość: 854.9099731445312
Procentowa jakość regresji: 98.19%

Przykład 8:
Przewidywana wartość: 800.1185913085938
Rzeczywista wartość: 813.02001953125
Procentowa jakość regresji

In [13]:
from torch_geometric.data import DataLoader
from torch.optim import Adam

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Definiuj DataLoader dla zbiorów uczącego i testowego
train_loader = DataLoader(train_dataset, batch_size=1, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=1, shuffle=False)

# Przykładowe użycie modelu
input_dim = 1  # Brak cech wierzchołków, jedna cecha domyślna
hidden_dim = 192  # liczba neuronów w warstwie ukrytej
output_dim = 1   # wymiar wyjścia (regresja)
num_layers = 3   # liczba warstw GINConv
dropout = 0.05    # prawdopodobieństwo dropoutu

# model = GINRegression(input_dim, hidden_dim, output_dim, num_layers, dropout).float().to(device)
model = GraphRegressionModel(dropout=dropout)
print(model)
optimizer = Adam(model.parameters(), lr=0.01)


GraphRegressionModel(
  (conv1): GraphConv(1, 64)
  (fc1): Linear(in_features=66, out_features=64, bias=True)
  (dropout): Dropout(p=0.05, inplace=False)
  (fc2): Linear(in_features=64, out_features=1, bias=True)
)


# Trening modelu


In [9]:
# Trenuj model
def train(epoch):
    model.train()
    print(len(train_loader))
    for data in train_loader:
        optimizer.zero_grad()
        output = model(data)
        #output = model(data.x, data.edge_index)
        loss = model.loss(output, data.y)
        loss.backward()
        optimizer.step()


# Ewaluacja modelu

In [10]:
def test(loader):
    model.eval()
    total_loss = 0
    for data in loader:
        output = model(data)
        #output = model(data.x, data.edge_index)
        total_loss += model.loss(output, data.y).item()
    return total_loss / len(loader)


# Wywołanie treningu i ewaluacji

In [11]:
# Przeprowadź trening i ewaluację
for epoch in range(1, 101):
    train(epoch)
    train_loss = test(train_loader)
    test_loss = test(test_loader)
    print(f'Epoch: {epoch}, Train Loss: {train_loss:.4f}, Test Loss: {test_loss:.4f}')

184


C:\Users\jderd\AppData\Local\Temp\ipykernel_21620\794946686.py:36: UserWarning: Using a target size (torch.Size([1, 1])) that is different to the input size (torch.Size([28, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(pred, score)


Epoch: 1, Train Loss: 51072.6552, Test Loss: 60262.2976
184
Epoch: 2, Train Loss: 52569.5933, Test Loss: 60528.7949
184
Epoch: 3, Train Loss: 49015.4461, Test Loss: 59581.5037
184
Epoch: 4, Train Loss: 51250.5943, Test Loss: 58801.5894
184
Epoch: 5, Train Loss: 49117.6971, Test Loss: 56942.9928
184
Epoch: 6, Train Loss: 51634.6756, Test Loss: 58249.3101
184
Epoch: 7, Train Loss: 45260.2548, Test Loss: 55248.4855
184
Epoch: 8, Train Loss: 45090.2410, Test Loss: 55438.6659
184
Epoch: 9, Train Loss: 45806.8905, Test Loss: 52549.7588
184
Epoch: 10, Train Loss: 48682.4704, Test Loss: 54181.5169
184
Epoch: 11, Train Loss: 39898.5331, Test Loss: 47531.2800
184
Epoch: 12, Train Loss: 39413.5759, Test Loss: 46080.1255
184
Epoch: 13, Train Loss: 36264.2882, Test Loss: 44161.8646
184
Epoch: 14, Train Loss: 35619.3404, Test Loss: 41650.8509
184
Epoch: 15, Train Loss: 32428.7488, Test Loss: 39838.1597
184
Epoch: 16, Train Loss: 30563.0840, Test Loss: 36055.5909
184
Epoch: 17, Train Loss: 28420.4653

In [26]:
sample_data = test_dataset[0]

# Przeprowadzenie predykcji na przykładowym obiekcie
model.eval()
with torch.no_grad():
    prediction = optimal_model(sample_data)[0][0].item()

# Wyświetlenie wyników
print("Przewidywana wartość:", prediction)
print("Rzeczywista wartość:", sample_data.y.item())

Przewidywana wartość: 656.6989135742188
Rzeczywista wartość: 874.5499877929688
